## Data Science Problem

If next year looks like this year, what are projected sales for next year and what can we do to change that?


## Objectives

Create a sales predictor model that allows you to try different variables and predict more accurate sales
<br><br>Will also want to track sales by sales channel/growth by sales channel

## 1 Imports

In [1]:
import os
import pandas as pd
import numpy as np

## 2 Load Data and convert to dataframe

In [2]:
sales_df = pd.read_csv('sales_2020-08-01_2023-09-06.csv')

In [3]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3228 entries, 0 to 3227
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order ID          3228 non-null   int64  
 1   Sale ID           3228 non-null   int64  
 2   Date              3228 non-null   object 
 3   Order             3228 non-null   object 
 4   Transaction type  3228 non-null   object 
 5   Sale type         3228 non-null   object 
 6   Sales channel     3228 non-null   object 
 7   POS location      679 non-null    object 
 8   Billing country   2549 non-null   object 
 9   Billing region    2542 non-null   object 
 10  Billing city      2549 non-null   object 
 11  Shipping country  2532 non-null   object 
 12  Shipping region   2525 non-null   object 
 13  Shipping city     2532 non-null   object 
 14  Product type      2010 non-null   object 
 15  Product vendor    2291 non-null   object 
 16  Product           2301 non-null   object 


In [4]:
pd.set_option('display.max_columns', None)

sales_df.head()

,Order ID,Sale ID,Date,Order,Transaction type,Sale type,Sales channel,POS location,Billing country,Billing region,Billing city,Shipping country,Shipping region,Shipping city,Product type,Product vendor,Product,Variant,Variant SKU,Net quantity,Gross sales,Discounts,Returns,Net sales,Shipping,Taxes,Total sales
0,2468022255679,7500145655871,2020-08-08T20:00:19-04:00,#1254,product,order,Online Store,NaN,United States,Pennsylvania,Pittsburgh,United States,Pennsylvania,Pittsburgh,Hand Dyed Yarn,Ex Libris Fibers,TEST,NaN,NaN,1,1.0,0.0,0.0,1.0,0.00,0.0,1.00
1,2468022255679,7500145688639,2020-08-08T20:00:19-04:00,#1254,shipping,order,Online Store,NaN,United States,Pennsylvania,Pittsburgh,United States,Pennsylvania,Pittsburgh,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,3.71,0.0,3.71
2,2468022255679,7500157288511,2020-08-08T20:03:42-04:00,#1254,product,return,Online Store,NaN,United States,Pennsylvania,Pittsburgh,United States,Pennsylvania,Pittsburgh,Hand Dyed Yarn,Ex Libris Fibers,TEST,NaN,NaN,-1,0.0,0.0,-1.0,-1.0,0.00,0.0,-1.00
3,2468022255679,7500157255743,2020-08-08T20:03:42-04:00,#1254,shipping,return,Online Store,NaN,United States,Pennsylvania,Pittsburgh,United States,Pennsylvania,Pittsburgh,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0.0,0.0,-3.71,0.0,-3.71
4,2469497241663,7505168695359,2020-08-09T12:04:57-04:00,#1255,product,order,Online Store,NaN,United States,Michigan,Redford,United States,Michigan,Redford,Hand Dyed Yarn,Ex Libris Fibers,My Abandonment -- Solnit Base (Superwash merin...,NaN,solnit-my-abandonment,1,23.0,0.0,0.0,23.0,0.00,0.0,23.00


## 3 Explore the Data

In [5]:
sales_df.shape

(3228, 27)

In [6]:
sales_df.dtypes

Order ID              int64
Sale ID               int64
Date                 object
Order                object
Transaction type     object
Sale type            object
Sales channel        object
POS location         object
Billing country      object
Billing region       object
Billing city         object
Shipping country     object
Shipping region      object
Shipping city        object
Product type         object
Product vendor       object
Product              object
Variant              object
Variant SKU          object
Net quantity          int64
Gross sales         float64
Discounts           float64
Returns             float64
Net sales           float64
Shipping            float64
Taxes               float64
Total sales         float64
dtype: object

### 3.1 Convert dates

In [7]:
#change dates to datetime, since whatever else I do I will want that formatted
sales_df['Date'] = pd.to_datetime(sales_df['Date'], utc=True)

sales_df['Date'].dtypes

datetime64[ns, UTC]

### 3.2 Identify key features

In [8]:
# what sales channels are being used?
sales_df['Sales channel'].unique()

array(['Online Store', 'Point of Sale', 'Shop'], dtype=object)

In [9]:
# vendor has three sales channels: Online Store (website), Point of Sale (purchases made in person at a yarn fair), 
# and Shop (purchases made at a brick-and-mortar store while being hosted as a visiting vendor)

sales_df['Sales channel'].replace({'Point of Sale' : 'Yarn Fair', 'Shop' : 'Shop Pop Up'}, inplace=True) 

sales_df['Sales channel'].unique()

array(['Online Store', 'Yarn Fair', 'Shop Pop Up'], dtype=object)

In [10]:
# relevant columns for me are: 
# 'Order ID' to see how many units/SKUs per order 
# 'Date'
# 'Transaction type' to sort shipping from products - check unknowns
# 'Sale type' to track orders and returns
# 'Sales channel' - 'Online Store' = website, 'Point of Sale' = trade show, 'Shop' = popup in a store
# 'Product type' - combine Gift Card and Gift Cards, check nans
# 'Product' - check nans
# 'Net quantity'
# 'Gross sales'
# 'Discounts'
# 'Returns'
# 'Net sales'
# 'Shipping'

In [11]:
# drop columns
sales_df = sales_df.drop(columns=['Sale ID', 'Order', 'POS location', 
       'Billing country', 'Billing region', 'Billing city', 'Shipping country', 'Shipping region', 'Shipping city',
       'Product vendor', 'Variant', 'Variant SKU', 'Taxes', 'Total sales'])

In [12]:
sales_df.select_dtypes('object')

,Transaction type,Sale type,Sales channel,Product type,Product
0,product,order,Online Store,Hand Dyed Yarn,TEST
1,shipping,order,Online Store,NaN,NaN
2,product,return,Online Store,Hand Dyed Yarn,TEST
3,shipping,return,Online Store,NaN,NaN
4,product,order,Online Store,Hand Dyed Yarn,My Abandonment -- Solnit Base (Superwash merin...
...,...,...,...,...,...
3223,product,order,Online Store,Hand Dyed Yarn,Marble Staircase -- Shelley Base (NSW Worsted)
3224,product,order,Online Store,Hand Dyed Yarn,Ordinary Evils -- Shirley Base (Fingering)
3225,product,order,Online Store,Yarn,Queer Classics Sets & Skeins
3226,shipping,order,Online Store,NaN,NaN


### 3.3 Identify Missing Values

In [13]:
sales_df.isna().sum()

Order ID               0
Date                   0
Transaction type       0
Sale type              0
Sales channel          0
Product type        1218
Product              927
Net quantity           0
Gross sales            0
Discounts              0
Returns                0
Net sales              0
Shipping               0
dtype: int64

In [14]:
# What *is* in the columns that are missing values?
for col in sales_df:
    if sales_df[col].isna().sum() > 0:
        print(col, sales_df[col].unique())

Product type ['Hand Dyed Yarn' nan 'Monthly Yarn Club' 'Gift Card' 'Notions'
 'Gift Cards' 'Yarn' 'Handknits' 'sticker']
Product ['TEST' nan
 'My Abandonment -- Solnit Base (Superwash merino/nylon; 4ply Fingering Weight)'
 'The Goldfinch --  Solnit Base (Superwash merino/nylon; 4ply Fingering Weight)'
 'Iron Lilac -- Carson Base (Superwash Merino; 4-ply DK Weight)'
 'Acorn -- Jane Base (Merino/Silk; 2-ply Fingering Weight)'
 'Wraith Tea No. 18 -- Solnit Base (Superwash Merino/nylon; 4ply Fingering Weight)'
 'You Burn Me -- Ursula Base (Merino/Yak/Nylon; 3-ply Fingering Weight)'
 'Jump Cannon -- Carson Base (Superwash Merino; 4-ply DK Weight)'
 'Exquisite Corpse -- Solnit Base (Superwash merino/nylon; 4ply Fingering Weight)'
 'Oathbreaker -- Solnit Base (Superwash Merino/Nylon; 4-ply Fingering Weight)'
 'Stained Glass Forest -- Solnit Base (Superwash Merino/nylon; 4ply Fingering Weight)'
 'Werelight -- Solnit Base (Superwash Merino/nylon; 4ply Fingering Weight)'
 'Leviathan -- Solnit Ba

In [15]:
sales_df['Product type'].unique()

array(['Hand Dyed Yarn', nan, 'Monthly Yarn Club', 'Gift Card', 'Notions',
       'Gift Cards', 'Yarn', 'Handknits', 'sticker'], dtype=object)

## 4 Cleanup

In [16]:
# investigate in dummy df
product_nan = sales_df[sales_df['Product type'].isna()]
product_nan.head()

,Order ID,Date,Transaction type,Sale type,Sales channel,Product type,Product,Net quantity,Gross sales,Discounts,Returns,Net sales,Shipping
1,2468022255679,2020-08-09 00:00:19+00:00,shipping,order,Online Store,NaN,NaN,0,0.0,0.0,0.0,0.0,3.71
3,2468022255679,2020-08-09 00:03:42+00:00,shipping,return,Online Store,NaN,NaN,0,0.0,0.0,0.0,0.0,-3.71
6,2469497241663,2020-08-09 16:04:57+00:00,shipping,order,Online Store,NaN,NaN,0,0.0,0.0,0.0,0.0,5.58
9,2469499076671,2020-08-09 16:05:59+00:00,shipping,order,Online Store,NaN,NaN,0,0.0,0.0,0.0,0.0,4.43
11,2469572509759,2020-08-09 16:38:24+00:00,shipping,order,Online Store,NaN,NaN,0,0.0,0.0,0.0,0.0,8.34


In [17]:
product_nan['Transaction type'].unique()

array(['shipping', 'unknown', 'tip', 'product'], dtype=object)

In [18]:
product_nan['Transaction type'].value_counts(ascending=False)

shipping    916
product     284
unknown      11
tip           7
Name: Transaction type, dtype: int64

In [19]:
product_nan.loc[(product_nan['Transaction type'] == 'unknown')]

,Order ID,Date,Transaction type,Sale type,Sales channel,Product type,Product,Net quantity,Gross sales,Discounts,Returns,Net sales,Shipping
321,3064403919023,2020-11-15 20:26:51+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,-5.60,-5.60,0.0
322,3064410276015,2020-11-15 20:27:36+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,-11.20,-11.20,0.0
535,3222346793135,2021-01-31 18:47:29+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,-6.00,-6.00,0.0
783,3752190509231,2021-04-16 12:00:38+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,-16.20,-16.20,0.0
1050,3979102355631,2021-07-17 10:30:18+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,-14.40,-14.40,0.0
2017,4802398322863,2022-10-12 17:44:54+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,10.18,10.18,0.0
2020,4802398322863,2022-10-18 13:30:23+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,-10.18,-10.18,0.0
2602,5145544589487,2023-03-31 15:47:42+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,-8.97,-8.97,0.0
2604,5145544589487,2023-03-31 15:47:42+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,8.97,8.97,0.0
2929,5265054335151,2023-06-01 18:11:07+00:00,unknown,return,Online Store,NaN,NaN,0,0.0,0.0,8.69,8.69,0.0


In [20]:
# If I replace Transaction type == 'unknown' with Transaction type == 'return', 
# I can use Transaction Type to fill in Product Type nans and possibly Product nans as well

product_nan['Transaction type'].replace('unknown', 'return', inplace=True)
product_nan['Transaction type'].unique()


C:\Users\miran\AppData\Local\Temp\ipykernel_5688\4230026193.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_nan['Transaction type'].replace('unknown', 'return', inplace=True)


array(['shipping', 'return', 'tip', 'product'], dtype=object)

In [21]:
product_nan.loc[(product_nan['Transaction type'] == 'product')]

,Order ID,Date,Transaction type,Sale type,Sales channel,Product type,Product,Net quantity,Gross sales,Discounts,Returns,Net sales,Shipping
1775,4789968502959,2022-08-26 22:06:55+00:00,product,order,Yarn Fair,NaN,Hemp,1,21.0,-3.15,0.0,17.85,0.0
1777,4789984329903,2022-08-26 22:18:41+00:00,product,order,Yarn Fair,NaN,Evil garden,1,13.0,0.00,0.0,13.00,0.0
1785,4790036922543,2022-08-26 23:02:17+00:00,product,order,Yarn Fair,NaN,Green hemp,1,21.0,0.00,0.0,21.00,0.0
1791,4790066380975,2022-08-26 23:27:32+00:00,product,order,Yarn Fair,NaN,Bookish sticker,1,3.0,0.00,0.0,3.00,0.0
1792,4790066380975,2022-08-26 23:27:32+00:00,product,order,Yarn Fair,NaN,Bury Me in Books and Yarn sticker,1,3.0,0.00,0.0,3.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3060,5369765789871,2023-08-01 15:40:01+00:00,product,order,Yarn Fair,NaN,Yarn Pick Ups,4,108.0,0.00,0.0,108.00,0.0
3095,5371475296431,2023-08-02 15:50:57+00:00,product,order,Online Store,NaN,Wraith Tea no. 55 -- Carson (DK),2,54.0,0.00,0.0,54.00,0.0
3096,5371475296431,2023-08-02 15:50:57+00:00,product,order,Online Store,NaN,Bookish sticker,1,4.0,0.00,0.0,4.00,0.0
3101,5378258174127,2023-08-06 23:05:16+00:00,product,order,Yarn Fair,NaN,Sale Bin,2,26.0,0.00,0.0,26.00,0.0


In [22]:
product_nan['Product type'] = np.where(product_nan['Product type'].isna(), product_nan['Transaction type'], 
                                       product_nan['Product type'])

product_nan['Product type'].unique()

C:\Users\miran\AppData\Local\Temp\ipykernel_5688\2168730952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_nan['Product type'] = np.where(product_nan['Product type'].isna(), product_nan['Transaction type'],


array(['shipping', 'return', 'tip', 'product'], dtype=object)

In [23]:
product_nan['Product'] = np.where(product_nan['Product'].isna(), product_nan['Transaction type'], 
                                       product_nan['Product'])
product_nan.isna().sum()

C:\Users\miran\AppData\Local\Temp\ipykernel_5688\3582615689.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_nan['Product'] = np.where(product_nan['Product'].isna(), product_nan['Transaction type'],


Order ID            0
Date                0
Transaction type    0
Sale type           0
Sales channel       0
Product type        0
Product             0
Net quantity        0
Gross sales         0
Discounts           0
Returns             0
Net sales           0
Shipping            0
dtype: int64

In [24]:
#after exploring, update data in sales_df
sales_df['Transaction type'].replace('unknown', 'return', inplace=True)
sales_df['Product type'] = np.where(sales_df['Product type'].isna(), sales_df['Transaction type'], 
                                       sales_df['Product type'])
sales_df['Product'] = np.where(sales_df['Product'].isna(), sales_df['Transaction type'], 
                                       sales_df['Product type'])

In [26]:
sales_df.isna().sum()

Order ID            0
Date                0
Transaction type    0
Sale type           0
Sales channel       0
Product type        0
Product             0
Net quantity        0
Gross sales         0
Discounts           0
Returns             0
Net sales           0
Shipping            0
dtype: int64

## 5 "Save As" 

In [25]:
# save dataframe without overwriting the original data in order to preserve integrity

os.makedirs('C:/Users/miran/Desktop/Ex Libris Fibers', exist_ok=True)  
sales_df.to_csv('C:/Users/miran/Desktop/Ex Libris Fibers/sales_df.csv')